## AFLOW.org: database layers

![](./images/db_layers.png)

## AFLOW REST-API

The AFLOW REST-API provides an means to retrieve data by interfacing directly with the database. Resources (e.g. properties, calculation details, files) are accessed via the AFLOW URL (AURL) which has the form:

```
<server>:AFLOWDATA/<Project_layer>/<set_layer>/<entry>/?<keyword>
```

Specific properties can be targeted using the keywords in the query string ```?<keyword>```. Below is a list of keywords ordered by type:


![](./images/api_keywords.png)

### Examples
The URL for the formation enthalpy per atom of the ternary Heusler structure T0001.A2BC with composition CU<sub>2</sub>TiZN is:

```
aflowlib.duke.edu/AFLOWDATA/LIB3_RAW/Cu_pvTi_svZn/T0001.A2BC/?enthalpy_formation_atom
```

While the full list of entries in the set Cu-Ti-ZN can be retrieved using:

```
aflowlib.duke.edu/AFLOWDATA/LIB3_RAW/Cu_pvTi_svZn/?aflowlib_entries 
```

In [ ]:
"""
    Example python script to fetch the space groups from the API for the CuTiZn alloy system.
"""
import json
from urllib.request import urlopen

SERVER = 'http://aflowlib.duke.edu/AFLOWDATA'
PROJECT = '/LIB3_RAW' # Project: Ternary alloys
SET = '/Cu_pvTi_svZn' # Set: CuTiZn alloy system
ENTRIES = '/?aflowlib_entries' # Keyword: List of entries in this alloy system

response = urlopen(SERVER+PROJECT+SET+ENTRIES).read().decode('utf-8') # Download list of entries with urlopen
entry_list = response.rstrip().split(',') # parse response body into a list

# Loop through entry list and fetch the spacegroup_relax for each entry
for entry in entry_list:
    entry_sg_url = SERVER + PROJECT + SET + '/' + entry + '/?spacegroup_relax' # spacegroup relax URL
    entry_sg = urlopen(entry_sg_url).read().decode('utf-8') # fetch spacegroup with urlopen
    print('{0:<12}   {1:>4}'.format(entry + ":", entry_sg))

### REST-API Exercises

1. Use the AFLOW REST-API to download the formation enthalpies for all of the entries in the ternary alloy system AlNiTi: set layer “AlNi_pvTi_sv”.

2. Use the AFLOW REST-API to download the full list of binary alloy systems that are available in the LIB2 project. How many alloys systems are available for download? (AURL: aflowlib.duke.edu:AFLOWDATA/LIB2_RAW/?aflowlib_entries)

3. Use the AFLOW REST-API to download the stoichiometries and space groups for all entries in an alloy system of your choice. How many entries are present in this system?

## AFLUX: AFLOW Search-API

**Aim:** Programatically expose the same functionality as our web search interface at http://aflow.org/advanced.php

- AFLUX enables search through the query string of the URL ```?<operator>``` by exposing several logical operators.
- Operators can be inter-property and/or intra-property.

### List of logical operators and AFLUX syntax
|Logical operator |      AFLUX syntax          |        Operator             |
| --------------  |:--------------------------:| ---------------------------:|
| ```<block>```   |    ```(``` and ```)```     |  Intra- and inter-property  |
| ```<AND>```     |          ```,```           |  Intra- and inter-property  |
| ```<OR>```      |          ```:```           |  Intra- and inter-property  |
| ```<NOT>```     |          ```!```           |       Intra-property        |
| ```<loose>```   |          ```*```           |       Intra-property        |
| ```<string>```  |          ```'```           |       Inter-property        |
| ```<mute>```    |          ```$```           |       Intra-property        |

- In AFLUX, operators are used to create a query. In our terminology a query is comprised of a **matchbook** and **directives**.

![](./images/AFLUX_URL.png)

### Matchbook:
- Materials keywords with arguments: ```<server>?species((Na:K),Cl),nspecies(2),Egap(1*,*5),energy_cell```.

### Directives:
- Used for formatting, moving between layers (catalog) and pagination.
- **format** takes args ```"json"``` and ```"html"```.
- **catalog** takes args ```"icsd"```, ```"lib1"```, ```"lib2"```, ...
- **paging** controls page, number of entries for page and sorting. By default entries are sorted in ascending order of first materials keyword. Order can be reversed by using a negative page number.


## AFLUX: examples

In [ ]:
"""
    Help text.
"""
from urllib.request import urlopen

helptext = urlopen('http://aflowlib.duke.edu/search/API/?').read().decode('utf-8')
print(helptext)

In [ ]:
"""
    List of matchbook keywords.
"""
import json
from urllib.request import urlopen
from pprint import pprint

schema = urlopen('http://aflowlib.duke.edu/search/API/?schema,format(json)').read().decode('utf-8')
schema_json = json.loads(schema)
pprint(schema_json)

In [ ]:
"""
    Get all entries with an electronic band gap of 0.6 eV with a tolerance of +/- 0.1 eV returned as a JSON.
"""

import json
from urllib.request import urlopen

api = 'http://aflowlib.duke.edu/search/API/?'
cmd = 'Egap(0.5*,*0.7),paging(0),format(json)'
entry = urlopen(api + cmd).read().decode('utf-8')
entry_json = json.loads(entry)
print(entry_json)

In [ ]:
"""
    Get the top 10 insulators with the highest shear modulus.
"""
import json
from urllib.request import urlopen

api = 'http://aflowlib.duke.edu/search/API/?'
search = 'ael_shear_modulus_vrh'
cmd = ',Egap(6*),paging(-1,10),format(json)'

entry = urlopen(api + search + cmd).read().decode('utf-8')
entry_json = json.loads(entry)
entry_list = [{'compound': entry_json[e]['compound'], search: entry_json[e][search]} for e in entry_json.keys()]
for i in range(10):
    print('{0:<10} {1:7.3f}'.format(entry_list[i]['compound'] + ':', float(entry_list[i][search])))

## AFLUX: execises
1. Use AFLUX to retrieve the formation enthalpies for all entries in a ternary alloy system of your choice.

2. Use AFLUX to retrieve the space group numbers of all materials that contain Cu and Ti but not V (the logical NOT operator is denoted by the exclamation mark ```!```).

3. Use AFLUX to retrieve the band gaps of all of the materials that have a calculated bulk modulus between 200 GPa and 300 GPa.

4. Count the materials in the ICSD catalog that contain the element Sn but not Pb, and that have a band gap between 1 eV and 3 eV.